<a href="https://colab.research.google.com/github/MichalMichniak/Sieci-Neuronowe/blob/main/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision
import torch
import torch.optim as optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)



100%|██████████| 26421880/26421880 [00:16<00:00, 1602340.12it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 235041.38it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 4291991.63it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 14688623.80it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [ ]:
print(train_dataloader.dataset.data.shape)


torch.Size([60000, 28, 28])
torch.Size([60000, 28, 28])


In [ ]:
class NN:
  def __init__(self):
    self.model = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Flatten(),
        nn.Linear(64 * 16 * 16, 512),
        nn.ReLU(),
        nn.Linear(512, 64)
    ).to(device)

  def forward(self, x):
    return self.model(x)

  def train_model(self, train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001):
    # Użycie GPU jeśli dostępne
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model

    # Zdefiniowanie optymalizatora i funkcji straty
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()  # Użycie CrossEntropyLoss dla klasyfikacji

    for epoch in range(num_epochs):
        model.train()  # Ustaw model w tryb treningowy
        running_loss = 0.0

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Walidacja na końcu każdej epoki
        validate_model(model, test_dataloader, device)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.4f}")




In [ ]:
print(NN().nn)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Flatten(start_dim=1, end_dim=-1)
  (6): Linear(in_features=16384, out_features=512, bias=True)
  (7): ReLU()
  (8): Linear(in_features=512, out_features=64, bias=True)
)


In [ ]:

# Funkcja do walidacji modelu
def validate_model(model, test_dataloader, device):
    model.eval()  # Ustaw model w tryb ewaluacji
    total = 0
    correct = 0

    with torch.no_grad():  # Wyłączenie obliczania gradientów
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)  # Wybór klasy z najwyższym prawdopodobieństwem

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

# Przykład użycia
if __name__ == "__main__":
    # Zakładając, że training_data i test_data są zdefiniowane
    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=64)

    model = NN()
    train_model(model, train_dataloader, test_dataloader, num_epochs=10, learning_rate=0.001)